# Maxar Image Availability Analysis

The Maxar image availability workflow takes as input a list of TerraFund project ids and returns as output a csv listing every project and how much of that project’s area has Maxar imagery coverage.

#### Workflow:
1. Pull info on project characteristics for the entire portfolio using the TerraMatch API
    - Repo/notebook: terrafund-portfolio-analysis/tm-api.ipynb
    - Input: list of TerraFund project IDs
    - Output: csv of all project features
2. Using the TM API csv, pull Maxar metadata
    - Repo/notebook: maxar-tools/decision-tree-metadata.ipynb and maxar-tools/src/decision_tree.py (? may need to change b/c of my additions to the acquire_metadata function)
    - Input: csv of project features
    - Output: csv of maxar metadata
3. Create imagery features (??)
    - Repo/notebook: terrafund-portfolio-analysis/maxar-img-avail.py
    - Input: csv of maxar metadata and csv of TM project features
    - Output: csv of project features and percent imagery coverage
4. Identify projects with 100% imagery coverage


In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import shape
from shapely.geometry import Polygon, Point
from shapely import union_all
import ast
from datetime import datetime, timedelta
import re
import os
import math
import requests
import yaml
import json
import pyproj
import sys
sys.path.append('../src/')
import image_availability as img
import process_api_results as clean
import decision_trees as tree
import tm_api_utils as api_request

%load_ext autoreload
%autoreload 2

### Parameters

In [ ]:
# File paths
tm_auth_path = '../secrets.yaml'
tm_staging_url = "https://api-staging.terramatch.org/research/v3/sitePolygons?"                 # use for testing queries
tm_prod_url = "https://api.terramatch.org/research/v3/sitePolygons?"                            # Use to pull data for analysis'
approved_projects = '../terrafund-portfolio-analyses/projects_all_approved_202501091214.csv'    # List of projects with approved polygons
feats = '../data/tm_api_TEST.csv'                                                               # Polygon metadata & geometries from TM API
maxar_feats = '/home/darby/github_repos/maxar-tools/data/tm_api_TEST.csv'                       # Polygon metadata & geometries from TM API saved to maxar-tools repo
maxar_md = '../data/imagery_availability/comb_img_availability_2025-02-26.csv'                  # Metadata for Maxar images corresponding to polygons

# Define thesholds
cloud_thresh = 50             # Threshold for removing cloudy imagery
off_nadir_thresh = 30         # Threshold for removing imagery too far off nadir
sun_elev_thresh = 30          # Threshold for removing imagery with too steep of a sun angle
img_count = 1                 # Threshold for identifying image availability
baseline_range = (-366, 0)    # Baseline window (1 year before plantstart date)
ev_range = (730, 1095)        # Early verification window (2-3 years after plant start date)

### Load & Preprocess Data
Inputs: 
- TM API csv
- Maxar metadata csv

In [ ]:
# # Load TM API polygons & convert to dataframe
# polygons = pd.read_csv(feats)
# polygons.columns = polygons.columns.str.lower()    # Enforce lowercase column names
# poly_df = pd.DataFrame(polygons)
# poly_df.columns

# # Rename columns
# poly_df = poly_df.rename(columns={'name': 'poly_name','geometry': 'poly_geom'})

# # Convert 'plantstart' column to a datetime
# poly_df['plantstart'] = pd.to_datetime(poly_df['plantstart'], errors='coerce')

In [ ]:
# Load TM API polygons and convert to a GeoDataFrame
polygons = pd.read_csv(feats)
polygons.columns = polygons.columns.str.lower()   # Enforce lowercase column names

# Rename 'name' and 'geometry' columns
poly_df = polygons.rename(columns={'name': 'poly_name', 'geometry': 'poly_geom'})  

# Convert 'plantstart' column to a datetime
poly_df['plantstart'] = pd.to_datetime(poly_df['plantstart'], errors='coerce')

# Convert stringified 'poly_geom' dictionaries into real dictionaries
poly_df['poly_geom'] = poly_df['poly_geom'].apply(lambda x: shape(ast.literal_eval(x)) if isinstance(x, str) else shape(x))

# Convert 'poly_geom' (polygon geometries) from WKT to Shapely objects
poly_df['poly_geom'] = poly_df['poly_geom'].apply(shape)

# Add a field for the polygon centroid
poly_df['poly_centroid'] = poly_df['poly_geom'].iloc[0].centroid

# Convert DataFrame to GeoDataFrame
poly_gdf = gpd.GeoDataFrame(poly_df, geometry='poly_geom', crs="EPSG:4326")

In [ ]:
print(poly_gdf.shape)
poly_gdf.head()
len(poly_gdf['poly_id'].unique())
poly_gdf

In [ ]:
# Load Maxar images metadata and convert to a GeoDataFrame
images = pd.read_csv(maxar_md)
print(images.columns)

# Select relevent columns
img_df = images[['title', 'project_id', 'poly_id', 'datetime', 'area:cloud_cover_percentage', 'eo:cloud_cover', 'area:avg_off_nadir_angle', 'view:sun_elevation', 'img_geom']]

# Convert 'datetime' column to a datetime and rename
img_df.loc[:, 'datetime'] = pd.to_datetime(img_df['datetime'], format='%Y-%m-%dT%H:%M:%S.%fZ', errors='coerce') # Convert to datetime type
img_df.loc[:, 'datetime'] = img_df['datetime'].apply(lambda x: x.replace(tzinfo=None) if pd.notna(x) else x)    # Remove time zone info
img_df = img_df.rename(columns={'datetime': 'img_date'})                                                        # Rename 'datetime' column 'img_date'

# Convert stringified 'poly_geom' dictionaries into real dictionaries
img_df['img_geom'] = img_df['img_geom'].apply(lambda x: shape(ast.literal_eval(x)) if isinstance(x, str) else shape(x))

# Convert 'img_geom' (image footprint geometries) from WKT to Shapely objects
img_df['img_geom'] = img_df['img_geom'].apply(shape)

# Add a field for the image centroid
img_df['img_centroid'] = img_df['img_geom'].iloc[0].centroid

# Convert DataFrame to GeoDataFrame
img_gdf = gpd.GeoDataFrame(img_df, geometry='img_geom', crs="EPSG:4326")

In [ ]:
print(img_gdf.shape)
img_gdf.head()

### Merge Images with Polygons
Inputs:
- poly_gdf: geodataframe of polygon metadata
- img_gdf: geodataframe of maxar image metadata

Outputs:
- merged: merged geodataframe of maxar image metadata + associated polygon metadata

In [ ]:
# Merge the image data with the polygon data (preserving image data rows and adding associated polygon attributes)
merged_gdf = img_gdf.merge(poly_gdf, on=['project_id', 'poly_id'], how='left')

# Ensure correct datetime format
merged_gdf['plantstart'] = pd.to_datetime(merged_gdf['plantstart'], errors='coerce')
merged_gdf['img_date'] = pd.to_datetime(merged_gdf['img_date'], errors='coerce')

In [ ]:
print(merged_gdf.shape)
print(merged_gdf.columns)
merged_gdf.head()

In [ ]:
# Summarize the number of images by project and polygon for merged (unfiltered) images
merged_gdf_summary = (
    merged_gdf.groupby(['project_id', 'poly_id'])
    .size()
    .reset_index(name='merged_img_count')
)

merged_gdf_summary

### Filter Images Based on Constraints
Inputs:
- merged: merged dataframe of maxar image metadata + associated polygon metadata

Outputs:
- filtered_merged: a filtered version of the merged dataframe of maxar image metadata + associated polygon metadata

In [ ]:
# Create a date differential column
merged_gdf['date_diff'] = (merged_gdf['img_date'] - merged_gdf['plantstart']).dt.days

# Filter to retain only images within the desired time range, cloud cover, off nadir angle, and sun elevation parameters
img_gdf_filtered = merged_gdf[
    (merged_gdf['date_diff'] >= baseline_range[0]) &
    (merged_gdf['date_diff'] <= baseline_range[1]) &
    (merged_gdf['area:cloud_cover_percentage'] < cloud_thresh) &
    (merged_gdf['area:avg_off_nadir_angle'] <= off_nadir_thresh) &
    (merged_gdf['view:sun_elevation'] >= sun_elev_thresh)
].copy()    # Copy to avoid SettingWithCopyWarning

In [ ]:
print('img_gdf_filtered Unique Polygons:', len(img_gdf_filtered['poly_id'].unique()))
img_gdf_filtered['poly_id'].value_counts()

In [ ]:
# Summarize  the number of images by project and polygon for filtered (by date and cloud cover) images
img_gdf_filtered_summary = (
    img_gdf_filtered.groupby(['project_id', 'poly_id'])
    .size()
    .reset_index(name='filtered_img_count')
)

img_gdf_filtered_summary

In [ ]:
# Check the # of images with different cloud cover percentages in the filtered imagery
print(img_gdf_filtered.shape)
img_gdf_filtered['area:cloud_cover_percentage'].value_counts()

### Compute Coverage for Each Polygon
Input:
- poly_gdf: geodataframe of polygon metadata
- img_gdf_filtered: a filtered version of the merged geodataframe of maxar image metadata + associated polygon metadata

Output:
- csv of percent imagery coverage by project

In [ ]:
# SCRATCH IGNORE
#  Empty list to hold results
results = []

for _, polygon in poly_gdf.iterrows():
    poly_id = polygon['poly_id']
    project_id = polygon['project_id']
    poly_geom = polygon['poly_geom'] # Geometry column
    poly_area = polygon['calcarea']  # Precomputed area of the polygon in hectares

    # Create a filtered GeoDataFrame that contains only images that intersect the polygon
    images = img_gdf_filtered[img_gdf_filtered["img_geom"].intersects(poly_geom)]

images
    

In [ ]:
# SCRATCH IGNORE
# Create empty dictionary to store poly_ids for each prj_id
project_polygons = {}

# Extract a list of the unique project_id values from shp_gdf   
prj_keys = list(set(poly_gdf.project_id))

for key in prj_keys:
    print('project key:', key)
    # Extract all poly_ids associated with this project_id
    poly_keys = poly_gdf[poly_gdf['project_id'] == key]['poly_id'].tolist()
    #print('poly keys:', poly_keys)

    # Store it in a dictionary
    project_polygons[key] = poly_keys

# Print to check
for key, polys in list(project_polygons.items()):
    print(f"Project ID: {key}, Polygon IDs: {polys}")

In [ ]:
# SCRATCH IGNORE
for project_id, poly_list in project_polygons.items():
    print(project_id, poly_list)

In [ ]:
# Vectorized option to select a best image per polygon from Rhiannon's MSU code. But can't add complex logic like combining two images in it
mins_metadata = img_gdf_filtered.loc[img_gdf_filtered.groupby('poly_id')['area:cloud_cover_percentage'].idxmin()].reset_index(drop=True)
mins_metadata.shape

### Reproject Polygons & Image Footprints

In [ ]:
def get_utm_crs(long, lat):
    """
    Determine the best UTM CRS based on polygon centroid location
    """
    utm_zone = int((long + 180) / 6) + 1
    hemisphere = 32600 if lat >= 0 else 32700 # Northern vs Southern hemisphere
    return f"EPSG:{hemisphere+utm_zone}"

In [ ]:
# Step 1: Create a dictionary mapping project_id --> list of poly_ids
project_polygons = poly_gdf.groupby("project_id")["poly_id"].apply(list).to_dict()

# Create an empty list to store
results = []

# Step 2: Iterate through each project_id
for project_id, poly_list in project_polygons.items():
    # First, filter img_gdf_filtered by 'project_id'
    project_images = img_gdf_filtered[img_gdf_filtered['project_id'] == project_id]
    print(f"There are {len(project_images)} images in the filtered image dataset associated with Project: {project_id}")

    # Step 3: Iterate through each polygon in this project
    for poly_id in poly_list:
        print(f"Checking polygon {poly_id}...")
        # Retrieve polygon geometry
        polygon_row = poly_gdf[poly_gdf['poly_id'] == poly_id].iloc[0]
        poly_geom = polygon_row["poly_geom"]
        poly_area = polygon_row["calcarea"] # For now, using precomputed area (in hectares)
        print(f"Polygon {poly_id}'s area is {poly_area} hectares")

        # Get the centroid of the polygon to determine the UTM zone
        #poly_centroid = poly_geom.centroid
        #print(f"Polygon Centroid for Polygon {poly_id} is {poly_centroid}")
        utm_crs = get_utm_crs(poly_centroid.x, poly_centroid.y) # Determine the best UTM CRS
        print('UTM CRS:', utm_crs)

        # Reproject the polygon to its correct UTM Zone
        poly_reprojected = gpd.GeoDataFrame([polygon_row], geometry=[poly_geom], crs="EPSG:4326").to_crs(utm_crs)
        poly_geom_reprojected = poly_reprojected.geometry.iloc[0] # Extract reprojected polygon

        # Now filter 'project_images' by 'poly_id'
        poly_images = project_images[project_images['poly_id'] == poly_id]
        print(f"Before reprojecting, there are {len(poly_images)} images in the filtered image dataset for Polygon {poly_id}")
        print(f"Original CRS: {poly_images.crs}")

        # If there are no images for this polygon, append that to the results list
        if poly_images.empty:
            # No imagery --> 0% coverage
            results.append((poly_id, project_id, poly_area, 0, 0, 0))
            continue

        # Reproject the images to the same UTM CRS
        #poly_images_reprojected = gpd.GeoDataFrame(poly_images, geometry=poly_images['img_geom'], crs='EPSG:4326').to_crs(utm_crs)
        # Remove invalid or empty geometries
        poly_images = poly_images[poly_images.is_valid]
        poly_images = poly_images[~poly_images.is_empty]

        # Reset index to prevent row loss
        poly_images = poly_images.reset_index(drop=True)        
        poly_images = poly_images.to_crs(utm_crs)
        print(f"After reprojecting, there are {len(images_reprojected)} reprojected images for Polygon {poly_id}")
        print(f"Reprojected CRS: {poly_images.crs}")

        # Step 4: Select best image(s) (lowest cloud cover OR union all images)
        sorted_images = poly_images_reprojected.sort_values(by='area:cloud_cover_percentage') # Sort by lowest cloud cover percentage
        best_image = sorted_images.iloc[0] # Take the 1 best image (for now)
        best_image_geom = best_image.geometry
        print(f"The best image for Polygon {poly_id} is {best_image['title']}")

        # Step 5: Compute actual coverage of polygon using the best image
        covered_area = best_image_geom.intersection(poly_geom_reprojected).area # In square meters (really?)
        convered_area_ha = covered_area / 10000 # Convert to hectares
        print(f"The calculated covered area for polygon {poly_id} is {convered_area_ha}")